# **Google Cloud para convertir discursos en texto**

Autor: Juan Pablo Restrepo Lopera, juanpablorestrepolopera@gmail.com

El siguiente notebook presenta como usé los servicios de Google Cloud para transcribir un discurso de Obama.

El primer paso consiste en habilitar una cuenta en Google Drive, luego activar el servicio de *Speech to text* para por último crear un *bucket* en en servicio *Storage*.

Luego de realizar lo anterior, cree la carpeta *temp* dentro de la carpeta de los notebooks del servicio Colab de Google Drive. Y dentro de esta cree la carpeta tbd_convert. En *tbd_convert* almacené un discurso de Barack Obama (https://www.youtube.com/watch?v=77pnVFLkUjM&ab_channel=CNN). Posteriormente, en *temp* almacene el archivo speech-to-text-api-key.json, el cual contiene las credenciales de Google Cloud, con esto último completé los preparativos para el código que se ejecutará en las iguientes lineas y que permitirá transcribir el discruso del expresidente de los Estados Unidos

**Librerias básicas**

In [ ]:
%pip install --upgrade google-cloud-speech

In [ ]:
%pip install --upgrade google-cloud-storage

In [ ]:
%pip install pydub
%pip install sox
!apt -qq install -y sox

In [1]:
from google.colab import drive
import subprocess
import os
from pydub import AudioSegment

**Credenciales**

In [2]:
drive.mount('/content/gdrive')
#ensure the file is accessible
!ls '/content/gdrive/My Drive/Colab Notebooks/temp'
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/content/gdrive/My Drive/Colab Notebooks/temp/speech-to-text-api-key.json'
#ensure the path is set correctly
!echo $GOOGLE_APPLICATION_CREDENTIALS

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'Google Cloud para convertir discursos en texto.ipynb'	 Speech_to_text.ipynb
 speech-to-text-api-key.json				 tbd_convert
/content/gdrive/My Drive/Colab Notebooks/temp/speech-to-text-api-key.json


**Convertimos el archivo .mp3 del discurso de Obama en un archivo MONO .wav**

In [3]:
# STEP 1: Convert .mp3 to .wav file
# STEP 2: Use subprocess + sox to convert .wav file into MONO .wav file, make sure sample_rate_hertz = 44100
import subprocess
import os
from pydub import AudioSegment
# dir
dirname = '/content/gdrive/My Drive/Colab Notebooks/temp/tbd_convert/'
# function that takes a file and converts to the mono version
def convert_audio(filename):
# files
  filenamesplit = os.path.splitext(filename)[0]
  filenamewav = filenamesplit + '.wav'
  print(filenamewav)
  newfilename = "mono_" + filenamewav
# convert mp3 to wav                                                            
  sound = AudioSegment.from_mp3(dirname + filename)
  sound.export(dirname + filenamewav, format="wav")
# convert wav to wav mono
  command2 = ['sox',dirname + filenamewav,'-c','1',dirname + newfilename]
  subprocess.Popen(command2)
for filename in os.listdir(dirname):
  print(filename)
  print("Starting...")
  convert_audio(filename)
  print("Completed!!")

Obama We're building Iron Man.mp3
Starting...
Obama We're building Iron Man.wav
Completed!!


**Llevamos el archivo MONO .wav al bucket en Google Cloud**

In [4]:
# STEP 3: Copies all files from temp/tbd_convert folder into Google Cloud Storage bucket
from google.colab import auth
from google.cloud import storage
auth.authenticate_user()
project_id = 'shaped-strata-305406'
!gcloud config set project {project_id}
!gsutil ls
bucket_name = 'sp2tx_bucket'
!gsutil -m cp -r /content/gdrive/My\ Drive/Colab\ Notebooks/temp/tbd_convert/mono* gs://{bucket_name}/

Updated property [core/project].
gs://sp2tx_bucket/
Copying file:///content/gdrive/My Drive/Colab Notebooks/temp/tbd_convert/mono_Obama We're building Iron Man.wav [Content-Type=audio/x-wav]...
-
Operation completed over 1 objects/4.2 MiB.                                      


**Hacemos la transcripción del discurso**

In [5]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/content/gdrive/My Drive/Colab Notebooks/temp/speech-to-text-api-key.json'
# Instantiates a client
from google.cloud import speech

client = speech.SpeechClient()

# The name of the audio file to transcribe
gcs_uri = "gs://sp2tx_bucket/mono_Obama We're building Iron Man.wav"

audio = speech.RecognitionAudio(uri=gcs_uri)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=44100,
    language_code="en-US",
)

# Detects speech in the audio file
response = client.recognize(config=config, audio=audio)

for result in response.results:
    print("Transcript: {}".format(result.alternatives[0].transcript))

Transcript: today I am joined by researchers who invent some of the most advanced metals on the planet
Transcript:  designers were modeling prototypes in the digital cloud
Transcript:  folks from the Pentagon who helped to support their work
Transcript:  basically I'm here to announce that we're building Iron Man
Transcript:  I'm going to blast off in a second
Transcript:  we've been listening to secret project we've been working out for a while
Transcript:  not really
Transcript:  baby
Transcript:  it's classified


**Nótese que la trancipción fue exitosa, el servicio de Speech to text solo se equivocó interpretando la palabra 'maybe' al final de la intervención de Obama, donde entendió 'baby'**

Nota: doy créditos a Mari Bolton (https://maribolton.work/converting-audio-speech-to-text-using-googles-speech-to-text-api-drive-cloud-storage-and-c8c2d2eafa99) por los códigos previos a la transcripción del discurso.

**IMPORTANCIA PARA TELEPERFORMANCE**

Teleperformance es una organización experta en la gestión de la experiencia del cliente, así pues, muchos de sus servicios involucran interacción directa con los usuarios de otras organizaciones, interacciones que en general corresponden a llamadas telefónicas.

Este último aspecto es el que conecta al ejemplo realizado en este notebook con los servicios de Teleperformance (además de los controles internos de calidad y satisfacción) puesto que una forma eficiente de realizar autorías de satisfacción de los clientes atendidos, de efectividad de los operadores que que asesoran a los usuarios vía llamada telefónica en representación de otra empresa, entre otros, es emplear algoritmos que permitan capturar información clave de las llamadas, que luego con otras herramientas como el análisis de sentimientos, la inteligencia de negocio, etc, permiten optimizar procesos aumentando la felicidad del cliente y la rentabilidad del negocio.